# implement line sweeps of params
- all params fixed expcet one, which is swept over range
- specifically looking at test accuracy as a function of one param


In [1]:
from matplotlib import pyplot as plt
from scipy.special import softmax
from itertools import product
import numpy as np
from utils import *
from model import *
import time
import seaborn as sns
sns.set_context('talk')

%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
## timestamp and dir for saving
tstamp = time.perf_counter_ns()
import os
os.mkdir('figures/scratch_folders/%i'%tstamp)

In [3]:
## default params
expargs = {
  'condition':'blocked',
  'n_train':160,
  'n_test':40
}
schargs = {
    'concentration':1.31,
    'stickiness_wi':2000,
    'stickiness_bt':9, # 100
    'sparsity':0.06,
    'pvar': 2,
    'lrate':1,
    'lratep':1,
    'decay_rate':1,
} 
semargs = {
  'beta2':False
}
args = {
    'sem':semargs,
    'sch':schargs,
    'exp':expargs
}
param_str = "-".join(["%s_%.3f"%(i,j) for i,j in schargs.items()])
param_str += "-"+"-".join(["%s_%.3f"%(i,j) for i,j in semargs.items()])
param_str

'concentration_1.310-stickiness_wi_2000.000-stickiness_bt_9.000-sparsity_0.060-pvar_2.000-lrate_1.000-lratep_1.000-decay_rate_1.000-beta2_0.000'

In [4]:
# p_name = 'concentration' # 
# p_vals = np.arange(0.1,2.5,0.2)

# p_name = 'stickiness_wi' # 5000
# p_vals = np.arange(0,6000,1000)

p_name = 'stickiness_bt' # 100
p_vals = np.arange(5,15,1)

# p_name = 'sparsity' # 0.08
# p_vals = np.arange(0.01,0.5,0.05)

# p_name = 'pvar' # 0
# p_vals = np.arange(1,2,0.1)

# p_name = 'lrate' # 1
# p_vals = np.arange(0.5,1,0.05)

# p_name = 'lratep' # 1
# p_vals = np.arange(0.8,1.05,0.01)

# p_name = 'decay_rate' # 1
# p_vals = np.arange(0.95,1.01,0.005)

p_name,p_vals

('stickiness_bt', array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14]))

### main

In [5]:
def pltsave_macc(macc,schargs=None,labL=['B','I','E','M','L'],close=True):
  """ 
  save accuracy of individual parameter setting 
  used in paramsearch loops
  """
  plt.figure(figsize=(10,4))
  ax=plt.gca()
  for idx in range(len(macc)):
    ax.plot(macc[idx],label=labL[idx])
  ax.axhline(0.5,c='k')
  plt.legend()
  param_str = "-".join(["%s_%.3f"%(i,j) for i,j in schargs.items()])
  plt.savefig('figures/scratch_folders/%i/acc-%s.jpg'%(tstamp,param_str))
  if close:
    plt.close('all')
  return None


In [6]:
ns = 25
dfL = []
condL = ['blocked','interleaved','early','middle','late']
for idx,p_val in enumerate(p_vals):
  print(idx/len(p_vals))
  
  args['sch'][p_name] = p_val  
  exp_batch_data = run_batch_exp_curr(ns,args,condL)
  ## acc
  batch_acc = unpack_acc(exp_batch_data) # curr,seeds,trials
  mean_acc = batch_acc.mean(1)
  test_acc = mean_acc[:,-40:].mean(1) # curr  
  
  ## save traces of EML for each param setting
  pltsave_macc(mean_acc[2:],args['sch'],labL=['E','M','L'])
  
  ## record
  gsD = {
    **schargs,
    **dict(zip(condL,test_acc))
  }
  dfL.append(gsD)
  
gsdf = pd.DataFrame(dfL)

0.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9


In [7]:
gsdf ## test acc dataframe

,concentration,stickiness_wi,stickiness_bt,sparsity,pvar,lrate,lratep,decay_rate,blocked,interleaved,early,middle,late
0,1.31,2000,5,0.06,2,1,1,1,0.91275,1.00000,0.89025,1.00000,0.97900
1,1.31,2000,6,0.06,2,1,1,1,0.92675,1.00000,0.89850,1.00000,0.98900
2,1.31,2000,7,0.06,2,1,1,1,0.88950,0.99125,0.87300,1.00000,1.00000
3,1.31,2000,8,0.06,2,1,1,1,0.87800,0.91900,0.87950,0.87825,0.90825
4,1.31,2000,9,0.06,2,1,1,1,0.87175,0.74375,0.89225,0.69200,0.62025
5,1.31,2000,10,0.06,2,1,1,1,0.94625,0.56500,0.88575,0.54775,0.62200
6,1.31,2000,11,0.06,2,1,1,1,0.90425,0.52825,0.87175,0.52600,0.51225
7,1.31,2000,12,0.06,2,1,1,1,0.90525,0.51200,0.89975,0.52175,0.51150
8,1.31,2000,13,0.06,2,1,1,1,0.83400,0.50075,0.91300,0.51675,0.50800
9,1.31,2000,14,0.06,2,1,1,1,0.94750,0.50375,0.91950,0.49150,0.47900


### plt 

In [8]:
plt.figure(figsize=(20,10))
ax = plt.gca()
gsdf.plot(p_name,condL,ax=ax)
ax.set_ylim(0.4,1.05)
ax.set_ylabel('test acc')
plt.title(param_str)
plt.savefig('figures/scratch/testacc-sweep_%s-default_%s-t%s.png'%(
  p_name,param_str,tstamp))